In [3]:
# Load necessary Packages
import os
import sys
import json
import pandas as pd

FILES = [
    'mailings.csv',
    'sent_email_events.csv',
    'sent_emails.csv',
    'session_events.csv',
    'sessions.csv'
]

ROOT_PATH = os.path.join(sys.path[0], '../raw_data/')

In [4]:
# Load up all files in memory, store in dict
file_paths = [os.path.join(ROOT_PATH, file) for file in FILES]

df_list = {}

for file_path in file_paths:
    df_name = os.path.splitext(os.path.basename(file_path))[0]
    temp_df = pd.read_csv(file_path)
    df_list[df_name] = temp_df


def data_checks(df: pd.DataFrame, key: str) -> None:
    '''
    Quick and dirty data checks. 
    
    '''

    print(f'---- {key} Data Checks ----')
    print(f'rows: {df.shape[0]}, columns: {df.shape[1]}')
    print(df.head())
    columns = df.columns.to_list()
    print(f'COLUMNS: {columns}')
    for col in columns:
        print(f'--- {col} --- ')
        col_temp = df[col]
        # check distinct values
        distinct_vals = len(col_temp.drop_duplicates())
        print(f'{distinct_vals} distinct values.')

        # see if length of DF is same as column values
        if df.shape[0] == distinct_vals:
            print(f'{col} could be PK or FK.')

        if distinct_vals < 20:
            print(f'{col} is likely categorical variable')
            print(col_temp.drop_duplicates().to_list())

        # check for nulls
        print(f'{col_temp.isna().sum()} nulls.')

        # data types
        print(f'{col_temp.dtype}')




#### mailings.csv
Represents the type of email sent for a specific day
 
Schema:
* mailing_id - primary key
* mailer - type of email
* action - subtype of email
* mailing_on - date of email send

In [5]:
df_mailings = df_list['mailings']
data_checks(df_mailings, 'mailings')

---- mailings Data Checks ----
rows: 2, columns: 4
   mailing_id        mailer                   action  mailing_on
0      297147  Notification       connection_request  2020-02-12
1      297161  Notification  received_recommendation  2020-02-12
COLUMNS: ['mailing_id', 'mailer', 'action', 'mailing_on']
--- mailing_id --- 
2 distinct values.
mailing_id could be PK or FK.
mailing_id is likely categorical variable
[297147, 297161]
0 nulls.
int64
--- mailer --- 
1 distinct values.
mailer is likely categorical variable
['Notification']
0 nulls.
object
--- action --- 
2 distinct values.
action could be PK or FK.
action is likely categorical variable
['connection_request', 'received_recommendation']
0 nulls.
object
--- mailing_on --- 
1 distinct values.
mailing_on is likely categorical variable
['2020-02-12']
0 nulls.
object


### sent_emails.csv
Represents a single email sent.
 
#### Schema:
* sent_email_id - the primary key
* mailing_id - mailings foreign key

In [6]:
df_sent_emails = df_list['sent_emails']

data_checks(df_sent_emails, 'sent_emails')

---- sent_emails Data Checks ----
rows: 76768, columns: 2
   sent_email_id  mailing_id
0     2898749029      297147
1     2898725736      297147
2     2898694715      297147
3     2898743292      297147
4     2898814800      297147
COLUMNS: ['sent_email_id', 'mailing_id']
--- sent_email_id --- 
76768 distinct values.
sent_email_id could be PK or FK.
0 nulls.
int64
--- mailing_id --- 
2 distinct values.
mailing_id is likely categorical variable
[297147, 297161]
0 nulls.
int64


### sent_email_events.csv
Represents the event stream of opens and clicks for a specific email
 
#### Schema:
* sent_email_id - sent_emails foreign key
* event_name - the specific event (either opened or clicked)
* properties - a json hash with the user agent of the browser in which the click occurred as well as a tracking_id that's linked to a session (only for click events)

In [7]:
df_sent_email_events = df_list['sent_email_events']
data_checks(df_sent_email_events, 'sent_email_events')


---- sent_email_events Data Checks ----
rows: 61611, columns: 3
   sent_email_id event_name                                         properties
0     2896858562    clicked  {"platform_name": "iOS (iPhone)", "platform_ve...
1     2896861134     opened                                                 {}
2     2896861157     opened                                                 {}
3     2896862637     opened                                                 {}
4     2896861968     opened                                                 {}
COLUMNS: ['sent_email_id', 'event_name', 'properties']
--- sent_email_id --- 
43738 distinct values.
0 nulls.
int64
--- event_name --- 
2 distinct values.
event_name is likely categorical variable
['clicked', 'opened']
0 nulls.
object
--- properties --- 
21997 distinct values.
0 nulls.
object


In [8]:
# verify that only "clicked" has properties json data
df_sent_email_events.loc[df_sent_email_events.event_name == 'opened', :]['properties'].drop_duplicates()

# check properties JSON in clicked emails
df_sent_email_clicked = df_sent_email_events.loc[df_sent_email_events.event_name == 'clicked', :]

json.loads(df_sent_email_clicked['properties'][0])

json_columns = ['platform_name', 'platform_version', 'browser_name', 'browser_version', 'device_name', 'tracking_id']

# copy dataframe to avoid index issues
df_sent_email_clicked_wide = df_sent_email_clicked.copy()

for column in json_columns: 
    df_sent_email_clicked_wide[column] = df_sent_email_clicked.apply(lambda row: json.loads(row['properties'])[column], axis=1)


In [9]:
# Set up subset dataframe to pull info out of click throughts

data_checks(df_sent_email_clicked_wide, 'df_sent_email_clicked_wide')

---- df_sent_email_clicked_wide Data Checks ----
rows: 21996, columns: 9
    sent_email_id event_name  \
0      2896858562    clicked   
6      2896859620    clicked   
9      2896866974    clicked   
10     2896867448    clicked   
14     2896867098    clicked   

                                           properties platform_name  \
0   {"platform_name": "iOS (iPhone)", "platform_ve...  iOS (iPhone)   
6   {"platform_name": "Android", "platform_version...       Android   
9   {"platform_name": "iOS (iPhone)", "platform_ve...  iOS (iPhone)   
10  {"platform_name": "Windows", "platform_version...       Windows   
14  {"platform_name": "iOS (iPhone)", "platform_ve...  iOS (iPhone)   

   platform_version browser_name browser_version device_name  \
0                13       Safari              13      iPhone   
6             8.0.0       Chrome              80     Unknown   
9                13       Safari              13      iPhone   
10             10.0       Chrome              79   

### sessions.csv
Represents the user session immediately after an email click occurred
 
#### Schema:
* session_id - primary key
* tracking_id - the link to a specific click event (from the sent_email_events.csv properties column)

In [10]:
df_sessions = df_list['sessions']
data_checks(df_sessions, 'sessions')

---- sessions Data Checks ----
rows: 21094, columns: 2
                                 session_id  \
0  b2604b5c-a942-4d86-84ea-28b445c53738       
1  5a8f2511-315a-4164-9f94-f267ac7304bd       
2  bdc5fc25-3880-410b-870d-743bcf2a6182       
3  aaebf8a6-1ff9-417f-8d23-2cae22981adf       
4  d765b594-a5c2-41fc-a172-fc149f9e5717       

                                tracking_id  
0  e607aee6-4e15-4212-8b71-c30f719c00fb      
1  2ddd0efb-ac38-4a4a-896f-bb31b82e0ba7      
2  9b704062-b9c9-4db6-93b5-aef226eb2fef      
3  228d9a6a-8f39-4f7c-98e7-f75833e5b46b      
4  8a1dc606-5539-4490-a9d2-97e303dd5ba5      
COLUMNS: ['session_id', 'tracking_id']
--- session_id --- 
21088 distinct values.
0 nulls.
object
--- tracking_id --- 
21082 distinct values.
0 nulls.
object


### session_events.csv
Represents events that occurred within a user session
 
#### Schema:
* session_id - session foreign key
* event_name - the name of the event

In [11]:
df_session_events = df_list['session_events']
data_checks(df_session_events, 'session_events')

---- session_events Data Checks ----
rows: 35767, columns: 2
                             session_id                   event_name
0  623481db-59ce-4991-9448-cef1a5b6b95e  connection_request_accepted
1  8be8a296-223a-478f-ace0-c042a6f1c887  connection_request_accepted
2  8be8a296-223a-478f-ace0-c042a6f1c887  connection_request_accepted
3  731ca44f-b30d-4e58-87a4-d9800f4c94ad  connection_request_accepted
4  2a7bbc93-249e-4a35-9b2a-33f6bcf7fc5f  connection_request_accepted
COLUMNS: ['session_id', 'event_name']
--- session_id --- 
17753 distinct values.
0 nulls.
object
--- event_name --- 
5 distinct values.
event_name is likely categorical variable
['connection_request_accepted', 'recommendation_request_accepted', 'connection_request_sent', 'conversation_message', 'recommendation_request_sent']
0 nulls.
object


In [12]:
# Check a few joins
df_mailings = df_list['mailings'].copy()
df_sent_emails = df_list['sent_emails'].copy()
df_sent_email_events = df_list['sent_email_events'].copy()
df_sessions = df_list['sessions'].copy()
df_session_events = df_list['session_events'].copy()

# fix data types for proper joins

df_sessions.session_id = df_sessions.session_id.astype(str)
df_session_events.session_id = df_session_events.session_id.astype(str)

# merge into wide boi DF
df = df_sent_emails.merge(df_mailings)
df = df.merge(df_sent_email_events, how='left')
df = df.merge(df_sent_email_clicked_wide, how='left')


# checking encoding as UTF-8 found there was a tab space at the end of the session ID causing joins to fail
df.tracking_id = df.tracking_id.astype(str).str.strip()
df_sessions.tracking_id = df_sessions.tracking_id.astype(str).str.strip()

df = df.merge(df_sessions, how='left', on='tracking_id')

df.session_id = df.session_id.astype(str).str.strip()
df_session_events.session_id = df_session_events.session_id.astype(str).str.strip()
df = df.merge(df_session_events, how='left', on='session_id')

df


,sent_email_id,mailing_id,mailer,action,mailing_on,event_name_x,properties,platform_name,platform_version,browser_name,browser_version,device_name,tracking_id,session_id,event_name_y
0,2898749029,297147,Notification,connection_request,2020-02-12,opened,{},NaN,NaN,NaN,NaN,NaN,nan,nan,NaN
1,2898749029,297147,Notification,connection_request,2020-02-12,clicked,"{""platform_name"": ""Windows"", ""platform_version...",Windows,10.0,Chrome,79,Unknown,7a2f1896-cbe1-4b80-af25-a3c98824d536,272db7b1-4132-4b01-bb1a-4d2107ec9bc3,connection_request_accepted
2,2898725736,297147,Notification,connection_request,2020-02-12,clicked,"{""platform_name"": ""Windows"", ""platform_version...",Windows,10.0,Internet Explorer,11,Unknown,3ea0c42c-3b44-4643-a6b2-17fc942d07d5,3c295d4b-7317-42f2-b51f-f79a5d0deab0,connection_request_accepted
3,2898725736,297147,Notification,connection_request,2020-02-12,opened,{},NaN,NaN,NaN,NaN,NaN,nan,nan,NaN
4,2898694715,297147,Notification,connection_request,2020-02-12,opened,{},NaN,NaN,NaN,NaN,NaN,nan,nan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112663,2896860449,297161,Notification,received_recommendation,2020-02-12,clicked,"{""platform_name"": ""Windows"", ""platform_version...",Windows,10.0,Chrome,80,Unknown,cb77de34-a00f-42ab-b100-774729c2a3b7,eb086811-1f36-458d-8d18-59d912813149,recommendation_request_sent
112664,2896860443,297161,Notification,received_recommendation,2020-02-12,opened,{},NaN,NaN,NaN,NaN,NaN,nan,nan,NaN
112665,2896858833,297161,Notification,received_recommendation,2020-02-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN
112666,2896858076,297161,Notification,received_recommendation,2020-02-12,opened,{},NaN,NaN,NaN,NaN,NaN,nan,nan,NaN


In [21]:
## Fix UUID issue, remove duplicates
df_sessions_clean = df_sessions.copy()
df_sessions_clean['session_id'] = df_sessions_clean['session_id'].str.strip()

df_sessions_clean = df_sessions_clean.drop_duplicates()
df_sessions_clean.to_csv(os.path.join(ROOT_PATH, 'sessions_cleaned.csv'), index=False)


df_session_events_clean = df_session_events.copy()
df_session_events_clean.to_csv(os.path.join(ROOT_PATH, 'session_events_cleaned.csv'), index=False)








In [22]:
df_sessions_clean.loc[df_sessions_clean.session_id == 'b8004189-772b-4516-b258-06c53dd4a717',:]

,session_id,tracking_id
1037,b8004189-772b-4516-b258-06c53dd4a717,cc1b55d9-83c9-4b1b-9973-ec33f1ef8346


In [237]:
# Quick check of the data across mailer
df.groupby(['mailing_id', 'action']).count()

,,sent_email_id,mailer,mailing_on,event_name_x,properties,platform_name,platform_version,browser_name,browser_version,device_name,tracking_id,session_id,event_name_y
mailing_id,action,,,,,,,,,,,,,
297147,connection_request,101794,101794,101794,71219,71219,35426,35393,35426,35426,35426,101794,101794,31571
297161,received_recommendation,10874,10874,10874,8419,8419,4597,4597,4597,4597,4597,10874,10874,4202


#### Initial Thoughts on Data

* No unexpected nulls found
* A couple duplicates found and removed
* Tab character in key required strip to join properly, need to fix before going to DB
* Next step set up a Postgres schema to verify keys and relationships



The schema should look like this:
mailing_id, mailer, action, sends, opens, clicks, connection_requests_accepted, connection_requests_sent, recommendations_accepted, recommendations_sent, conversation_messages_sent